In [44]:
!ls

'CRF For Bangla.ipynb'	  N.inb         bengali-stemmer   nlp_hackathon_bd_2023
'CRF for English.ipynb'   NLP.ipynb     ner.csv
 Getting_Started.ipynb	  archive.zip   ner_dataset.csv


In [45]:
def getDatasetFromIBOFile(filename): #nlp_hackathon_bd_2023/data/dev.txt"
    f = open(filename, "r")
    content = f.read()
    
    lines = content.split("\n")
    
    dataset = []
    for line in lines:
        parts = line.split(" _ _ ")
        word = parts[0]
        label = parts[len(parts)-1]

        dataset.append([word,label])
    return dataset


In [46]:
dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/dev.txt")
print(dataset[:30])

[['তিনি', 'O'], ['যুবক', 'O'], ['হিসেবে', 'O'], ['শেফিল্ড', 'B-GRP'], ['বুধবার', 'I-GRP'], ['এফ.সি.', 'I-GRP'], ['যোগদান', 'O'], ['করেন', 'O'], ['এবং', 'O'], ['১৯৫১', 'O'], ['সালে', 'O'], ['তাদের', 'O'], ['পেশাগত', 'O'], ['দিক', 'O'], ['দিয়ে', 'O'], ['আত্মপ্রকাশ', 'O'], ['করেন।', 'O'], ['', ''], ['ভিনেগার', 'O'], ['মাঝে', 'O'], ['মাঝে', 'O'], ['চাটনি', 'B-PROD'], ['ব্যবহার', 'O'], ['করা', 'O'], ['হয়।', 'O'], ['', ''], ['মূলত,', 'O'], ['দুর্গটি', 'O'], ['ছিল', 'O'], ['একটি', 'O']]


In [47]:
!pip install bltk

Keyring is skipped due to an exception: 'keyring.backends'


In [48]:
from bltk.langtools import PosTagger
pos_tagger = PosTagger()

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [49]:
# Convert data into features

feature_extracted_dataset = []
sentence = []
for row in dataset:
    word = row[0]
    label = row[1]
    if word=='':
        feature_extracted_dataset.append(sentence)
        sentence = []
        continue
    basic_word_features = (word, pos_tagger.pos_tag([word])[0][1], label)
    sentence.append(basic_word_features)
    

In [50]:
print(feature_extracted_dataset[:5])

[[('তিনি', 'PPR', 'O'), ('যুবক', 'NC', 'O'), ('হিসেবে', 'PP', 'O'), ('শেফিল্ড', 'NP', 'B-GRP'), ('বুধবার', 'NC', 'I-GRP'), ('এফ.সি.', 'NP', 'I-GRP'), ('যোগদান', 'NC', 'O'), ('করেন', 'VM', 'O'), ('এবং', 'CCD', 'O'), ('১৯৫১', 'RDX', 'O'), ('সালে', 'NC', 'O'), ('তাদের', 'PPR', 'O'), ('পেশাগত', 'JJ', 'O'), ('দিক', 'NC', 'O'), ('দিয়ে', 'VM', 'O'), ('আত্মপ্রকাশ', 'NC', 'O'), ('করেন।', 'VM', 'O')], [('ভিনেগার', 'NC', 'O'), ('মাঝে', 'NST', 'O'), ('মাঝে', 'NST', 'O'), ('চাটনি', 'NC', 'B-PROD'), ('ব্যবহার', 'NC', 'O'), ('করা', 'NV', 'O'), ('হয়।', 'VM', 'O')], [('মূলত,', 'NC', 'O'), ('দুর্গটি', 'NC', 'O'), ('ছিল', 'VM', 'O'), ('একটি', 'JQ', 'O'), ('সুরক্ষিত', 'JJ', 'O'), ('দুর্গ,', 'PU', 'O'), ('যা', 'PRL', 'O'), ('পোলিশ', 'NC', 'B-LOC'), ('-লিথুয়ানিয়ান', 'NC', 'I-LOC'), ('কমনওয়েলথ', 'NP', 'I-LOC'), ('এর', 'PPR', 'O'), ('দক্ষিণ', 'JJ', 'O'), ('সীমান্ত', 'NC', 'O'), ('রক্ষা', 'NC', 'O'), ('করে।', 'VM', 'O')], [('হার্নান্দেজ', 'NC', 'O'), ('আলোকচিত্রগ্রাহী', 'NC', 'B-PROD'), ('ফিল্ম', 'NP', 'I

lets Add some cool features now

In [77]:
def count_punctuations(text):
    punctuations='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    cnt = 0
    for i in punctuations:
        if i in text:
            cnt += 1
    return cnt
print(count_punctuations("মূলত-প্রশংসা"))

1


In [78]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        #'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        #'word.isupper()': word.isupper(),
        #'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        "containPunctuation": count_punctuations(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            #'-1:word.lower()': word1.lower(),
            #'-1:word.istitle()': word1.istitle(),
            #'-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            #'+1:word.lower()': word1.lower(),
            #'+1:word.istitle()': word1.istitle(),
            #'+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [79]:
X = [sent2features(s) for s in feature_extracted_dataset]
y = [sent2labels(s) for s in feature_extracted_dataset]

In [83]:
X[:2]

[[{'bias': 1.0,
   'word[-3:]': 'িনি',
   'word[-2:]': 'নি',
   'word.isdigit()': False,
   'postag': 'PPR',
   'postag[:2]': 'PP',
   'containPunctuation': 0,
   'BOS': True,
   '+1:postag': 'NC',
   '+1:postag[:2]': 'NC'},
  {'bias': 1.0,
   'word[-3:]': 'ুবক',
   'word[-2:]': 'বক',
   'word.isdigit()': False,
   'postag': 'NC',
   'postag[:2]': 'NC',
   'containPunctuation': 0,
   '-1:postag': 'PPR',
   '-1:postag[:2]': 'PP',
   '+1:postag': 'PP',
   '+1:postag[:2]': 'PP'},
  {'bias': 1.0,
   'word[-3:]': 'েবে',
   'word[-2:]': 'বে',
   'word.isdigit()': False,
   'postag': 'PP',
   'postag[:2]': 'PP',
   'containPunctuation': 0,
   '-1:postag': 'NC',
   '-1:postag[:2]': 'NC',
   '+1:postag': 'NP',
   '+1:postag[:2]': 'NP'},
  {'bias': 1.0,
   'word[-3:]': 'ল্ড',
   'word[-2:]': '্ড',
   'word.isdigit()': False,
   'postag': 'NP',
   'postag[:2]': 'NP',
   'containPunctuation': 0,
   '-1:postag': 'PP',
   '-1:postag[:2]': 'PP',
   '+1:postag': 'NC',
   '+1:postag[:2]': 'NC'},
  {'bi

In [84]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [86]:
!pip install sklearn

Keyring is skipped due to an exception: 'keyring.backends'


In [88]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report


In [90]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)


              precision    recall  f1-score   support

      B-CORP       0.36      0.16      0.22       127
        B-CW       0.10      0.03      0.04       120
       B-GRP       0.47      0.34      0.39       118
       B-LOC       0.40      0.23      0.29       101
       B-PER       0.48      0.43      0.46       144
      B-PROD       0.39      0.15      0.22       189
      I-CORP       0.46      0.26      0.33       122
        I-CW       0.24      0.07      0.11       161
       I-GRP       0.58      0.44      0.50       226
       I-LOC       0.23      0.11      0.15        61
       I-PER       0.51      0.45      0.48       180
      I-PROD       0.36      0.21      0.27       128
           O       0.91      0.98      0.94      8648

    accuracy                           0.86     10325
   macro avg       0.42      0.30      0.34     10325
weighted avg       0.82      0.86      0.84     10325



In [91]:
import eli5
eli5.show_weights(crf, top=30)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


TypeError: 'NoneType' object is not iterable